In [1]:
import gzip
import numpy
import random
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before
allRatings = []
userRatings = defaultdict(list)
data = list(readGz("Assignment1/HW3/assignment1/train.json.gz"))

In [17]:
# initialzation
users_perItem = defaultdict(list)
items_perUser = defaultdict(list)
userItem_rating = {} # rating for a specific user with respect to a specific item
user_bias = {}
item_bias = {}
user_gamma = defaultdict(list)
item_gamma = defaultdict(list)
alpha = 0
lam = 6
gamma_size = 2 #number of columns ina gamma

prev_MSE = 20 #randomly initializing previous MSE
MSE = 10 #randomly intitializing current MSE

# store the parameters that result in smallest MSE
user_bias_final = {}
item_bias_final = {}
user_gamma_final = defaultdict(list)
item_gamma_final = defaultdict(list)
alpha_final = 0

for l in data:#[:100000]:
    user,item = l['reviewerID'],l['itemID']
    users_perItem[item].append(l['reviewerID'])
    items_perUser[user].append(l['itemID'])
    userItem_rating[user+item] = l['rating']
    user_bias[user] = 0
    item_bias[item] = 0
for u in user_bias: # initialize gamma_u to be n by k
    for x in range(0,gamma_size):
        user_gamma[u].append(random.random())
for i in item_bias: # initialize gamma_i to be m by k
    for x in range(0,gamma_size):
        item_gamma[i].append(random.random())

In [10]:
while abs(MSE - prev_MSE) > 0.01:
    prev_MSE = MSE
    ## update user_gamma (user preference)
    for u in user_bias:
        for k in range(0,gamma_size):
            temp = []
            for i in items_perUser[u]:
                inner_product = sum([item_gamma[i][n]*user_gamma[u][n]*(n!=k) for n in range(0,gamma_size)])
                temp.append(item_gamma[i][k]*(userItem_rating[u+i] - alpha - user_bias[u] - item_bias[i] - inner_product))
            user_gamma[u][k] = sum(temp) / (sum([item_gamma[i][k]**2 for i in items_perUser[u]]) + lam)
    
                
    ## update item_gamma (item properties)
    for i in item_bias:
        for k in range(0,gamma_size):
            temp = []
            for u in users_perItem[i]:
                inner_product = sum([item_gamma[i][n]*user_gamma[u][n]*(n!=k) for n in range(0,gamma_size)])
                temp.append(user_gamma[u][k]*(userItem_rating[u+i] - alpha - user_bias[u] - item_bias[i] - inner_product))
            item_gamma[i][k] = sum(temp) / (sum([user_gamma[u][k]**2 for u in users_perItem[i]]) + lam)
            
    ## update alpha
    temp = []
    for l in data:#[:100000]:
        gamma_inner = sum([item_gamma[l['itemID']][n]*user_gamma[l['reviewerID']][n] for n in range(0,gamma_size)])
        temp.append(l['rating'] - (user_bias[l['reviewerID']] + item_bias[l['itemID']] + gamma_inner))
    alpha = sum(temp) / 200000#100000
    
    ## update user bias
    for u in user_bias:
        temp = []
        for i in items_perUser[u]:
            gamma_inner = sum([item_gamma[i][n]*user_gamma[u][n] for n in range(0,gamma_size)])
            temp.append(userItem_rating[u+i] - (alpha + item_bias[i] + gamma_inner))
        user_bias[u] = sum(temp) / (lam + len(items_perUser[u]))
        
    ## update item bias
    for i in item_bias:
        temp = []
        for u in users_perItem[i]:
            gamma_inner = sum([item_gamma[i][n]*user_gamma[u][n] for n in range(0,gamma_size)])
            temp.append(userItem_rating[u+i] - (alpha + user_bias[u] + gamma_inner))
        item_bias[i] = sum(temp) / (lam + len(users_perItem[i]))
        
            
    ## compute MSE
    diff = []
    for l in data:#[:100000]:
        gamma_inner = sum([item_gamma[l['itemID']][n]*user_gamma[l['reviewerID']][n] for n in range(0,gamma_size)])
        diff.append((l['rating'] - (alpha + user_bias[l['reviewerID']] + item_bias[l['itemID']] + gamma_inner))**2)
    MSE = sum(diff) / len(diff)
    
    print 'previous MSE for training = ', prev_MSE
    print 'current MSE for training = ', MSE
    
    
    if MSE < prev_MSE:
        ## update final results
        for u in user_bias:
            user_gamma_final[u] = [user_gamma[u][k] for k in range(0, gamma_size)]
            user_bias_final[u] = user_bias[u]
        for i in item_bias:
            item_gamma_final[i] = [item_gamma[i][k] for k in range(0, gamma_size)]
            item_bias_final[i] = item_bias[i]
        alpha_final = alpha

In [8]:
MSE
prev_MSE

0.8175521601885533

In [11]:
## Validation MSE
diff = []
for l in data[100000:200000]:
    if l['reviewerID'] not in user_bias_final and l['itemID'] not in item_bias_final:
        diff.append((l['rating'] - alpha_final)**2)
    elif l['reviewerID'] not in user_bias:
        diff.append((l['rating'] - (alpha_final + item_bias_final[l['itemID']]))**2)
    elif l['itemID'] not in item_bias:
        diff.append((l['rating'] - (alpha_final + user_bias_final[l['reviewerID']]))**2)
    else:
        gamma_inner = sum([item_gamma_final[l['itemID']][n]*user_gamma_final[l['reviewerID']][n] for n in range(0,gamma_size)])
        diff.append((l['rating'] - (alpha_final + user_bias_final[l['reviewerID']] + item_bias_final[l['itemID']] + gamma_inner))**2)
MSE = sum(diff) / len(diff)

print 'MSE = ', MSE,'for validating set (lambda = 1)'

MSE =  0.705643810621 for validating set (lambda = 1)


In [20]:
prev_MSE = 10 #randomly initializing previous MSE
MSE = 9 #randomly intitializing current MSE
while MSE < prev_MSE:#abs(prev_MSE - MSE) > 0.00001:
    prev_MSE = MSE
    ## update alpha
    alpha = [l['rating'] - (user_bias[l['reviewerID']] + item_bias[l['itemID']]) for l in data]#[:100000]]
    alpha = sum(alpha) / 200000#100000
    
    ## update user bias
    for u in user_bias:
        temp = sum([userItem_rating[u+i] - (alpha + item_bias[i]) for i in items_perUser[u]])
        user_bias[u] = temp / (lam + len(items_perUser[u]))
    
    ## update item bias
    for i in item_bias:
        temp = sum([userItem_rating[u+i] - (alpha + user_bias[u]) for u in users_perItem[i]])
        item_bias[i] = temp / (lam + len(users_perItem[i]))
        
    ## compute MSE
    diff = [(l['rating'] - (alpha + user_bias[l['reviewerID']] + item_bias[l['itemID']]))**2 for l in data]#[:100000]]
    MSE = sum(diff) / len(diff)

    
    if MSE < prev_MSE:
        ## update final results
        for u in user_bias:
            user_bias_final[u] = user_bias[u]
        for i in item_bias:
            item_bias_final[i] = item_bias[i]
        alpha_final = alpha

In [22]:
prev_MSE = 10 #randomly initializing previous MSE
MSE = 9 #randomly intitializing current MSE
while abs(prev_MSE - MSE) > 0.1:
    prev_MSE = MSE
    ## update user_gamma (user preference)
    for u in user_bias:
        for k in range(0,gamma_size):
            temp = []
            for i in items_perUser[u]:
                inner_product = sum([item_gamma[i][n]*user_gamma[u][n]*(n!=k) for n in range(0,gamma_size)])
                temp.append(item_gamma[i][k]*(userItem_rating[u+i] - alpha_final - user_bias_final[u] - item_bias_final[i] - inner_product))
            user_gamma[u][k] = sum(temp) / (sum([item_gamma[i][k]**2 for i in items_perUser[u]]) + lam)
    ## update item_gamma
    for i in item_bias:
        for k in range(0,gamma_size):
            temp = []
            for u in users_perItem[i]:
                inner_product = sum([item_gamma[i][n]*user_gamma[u][n]*(n!=k) for n in range(0,gamma_size)])
                temp.append(user_gamma[u][k]*(userItem_rating[u+i] - alpha_final - user_bias_final[u] - item_bias_final[i] - inner_product))
            item_gamma[i][k] = sum(temp) / (sum([user_gamma[u][k]**2 for u in users_perItem[i]]) + lam)
    
    ## compute MSE
    diff = []
    for l in data:#[:100000]:
        gamma_inner = sum([item_gamma[l['itemID']][n]*user_gamma[l['reviewerID']][n] for n in range(0,gamma_size)])
        diff.append((l['rating'] - (alpha_final + user_bias_final[l['reviewerID']] + item_bias_final[l['itemID']] + gamma_inner))**2)
    MSE = sum(diff) / len(diff)
    
    print 'previous MSE for training = ', prev_MSE
    print 'current MSE for training = ', MSE
    
    if MSE < prev_MSE:
        ## update final results
        for u in user_bias:
            user_gamma_final[u] = [user_gamma[u][k] for k in range(0, gamma_size)]
        for i in item_bias:
            item_gamma_final[i] = [item_gamma[i][k] for k in range(0, gamma_size)]

previous MSE for training =  9
current MSE for training =  0.759689242861


previous MSE for training =  0.759689242861
current MSE for training =  0.747976227951


In [162]:
## Validation MSE
diff = []
for l in data[100000:200000]:
    if l['reviewerID'] not in user_bias and l['itemID'] not in item_bias:
        diff.append((l['rating'] - alpha)**2)
    elif l['reviewerID'] not in user_bias:
        diff.append((l['rating'] - (alpha + item_bias[l['itemID']]))**2)
    elif l['itemID'] not in item_bias:
        diff.append((l['rating'] - (alpha + user_bias[l['reviewerID']]))**2)
    else:
        diff.append((l['rating'] - (alpha + user_bias[l['reviewerID']] + item_bias[l['itemID']]))**2)
MSE = sum(diff) / len(diff)

print 'MSE = ', MSE,'for validating set (lambda = 1)'

previous MSE for training =  9
current MSE for training =  2.4471268201


previous MSE for training =  2.4471268201
current MSE for training =  2.44716278922


In [49]:
features = []
features_user = []
features_item = []
features_alpha = []
ratings = []
for l in data:#[:100000]:
    base = alpha_final + user_bias_final[l['reviewerID']] + item_bias_final[l['itemID']]
    words_num = len(l['reviewText'].split()) #split the words
    ratings.append(l['rating'])
    if 'price' in l:
        features.append([base, words_num, l['price'], l['helpful']['outOf']])
    else:
        features.append([base, words_num, 0, l['helpful']['outOf']])

theta,residuals,rank,s = numpy.linalg.lstsq(features, ratings)

ratings = []
for l in data:#[:100000]:
    base = alpha_final
    words_num = len(l['reviewText'].split()) #split the words
    ratings.append(l['rating'])
    if 'price' in l:
        features_alpha.append([base, words_num, l['price'], l['helpful']['outOf']])
    else:
        features_alpha.append([base, words_num, 0, l['helpful']['outOf']])

theta_alpha,residuals,rank,s = numpy.linalg.lstsq(features_alpha, ratings)

ratings = []
for l in data:#[:100000]:
    base = alpha_final + user_bias_final[l['reviewerID']]
    words_num = len(l['reviewText'].split()) #split the words
    ratings.append(l['rating'])
    if 'price' in l:
        features_user.append([base, words_num, l['price'], l['helpful']['outOf']])
    else:
        features_user.append([base, words_num, 0, l['helpful']['outOf']])

theta_user,residuals,rank,s = numpy.linalg.lstsq(features_user, ratings)

ratings = []
for l in data:#[:100000]:
    base = alpha_final + item_bias_final[l['itemID']]
    words_num = len(l['reviewText'].split()) #split the words
    ratings.append(l['rating'])
    if 'price' in l:
        features_item.append([base, words_num, l['price'], l['helpful']['outOf']])
    else:
        features_item.append([base, words_num, 0, l['helpful']['outOf']])

theta_item,residuals,rank,s = numpy.linalg.lstsq(features_item, ratings)

In [50]:
theta

array([  1.02311919e+00,  -1.20516499e-03,   4.06168287e-04,
        -1.80933813e-03])

In [52]:
userItemRating = []

predictions = open("Assignment1/HW3/assignment1/predictions_Rating.txt", 'w')
for l in open("Assignment1/HW3/assignment1/pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u, i = l.strip().split('-')
    if u not in user_bias_final and i not in item_bias_final:
        predicted_rating = alpha_final
        if 'price' in l:
            feat_row = [predicted_rating, words_num, l['price'], l['helpful']['outOf']]
        else:
            feat_row = [predicted_rating, words_num, 0, l['helpful']['outOf']]
        predicted_rating = [theta_alpha[i]*feat_row[i] for i in range(0,len(feat_row))]
    elif u not in user_bias_final:
        predicted_rating = alpha_final + item_bias_final[i]
        if 'price' in l:
            feat_row = [predicted_rating, words_num, l['price'], l['helpful']['outOf']]
        else:
            feat_row = [predicted_rating, words_num, 0, l['helpful']['outOf']]
        predicted_rating = [theta_item[i]*feat_row[i] for i in range(0,len(feat_row))]
    elif i not in item_bias_final:
        predicted_rating = alpha_final + user_bias_final[u]
        if 'price' in l:
            feat_row = [predicted_rating, words_num, l['price'], l['helpful']['outOf']]
        else:
            feat_row = [predicted_rating, words_num, 0, l['helpful']['outOf']]
        predicted_rating = [theta_user[i]*feat_row[i] for i in range(0,len(feat_row))]
    else:
        #gamma_inner = sum([item_gamma_final[i][n]*user_gamma_final[u][n] for n in range(0,gamma_size)])
        predicted_rating = alpha_final + user_bias_final[u] + item_bias_final[i]# + gamma_inner
        if 'price' in l:
            feat_row = [predicted_rating, words_num, l['price'], l['helpful']['outOf']]
        else:
            feat_row = [predicted_rating, words_num, 0, l['helpful']['outOf']]
        predicted_rating = [theta[i]*feat_row[i] for i in range(0,len(feat_row))]
        
    if predicted_rating > 5.0:
        predictions.write(u + '-' + i + ',' + str(5.0) + '\n')
    else:
        predictions.write(u + '-' + i + ',' + str(predicted_rating) + '\n')
            
predictions.close()

TypeError: string indices must be integers, not str